In [3]:
#cd ..

c:\Users\TEJAS\OneDrive\Desktop\CRM-main - layerwise


In [4]:
from crm.core import Network, Decoder_fullyconnected

C:\Users\TEJAS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-28 17:15:11,943	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-28 17:15:13,513	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [14]:
#pip install ray

In [5]:
num_neurons = 8
adj_list = {0: [1, 2], 1: [3, 4], 2: [3, 4], 3: [5, 6, 7], 4: [5, 6, 7], 5: [], 6: [], 7: []}
crm_network = Network(num_neurons, adj_list)

In [18]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() and args.gpu else "cpu")


In [22]:
decoder = Decoder_fullyconnected(crm_network)
decoder.to(device)

Decoder_fullyconnected()

In [25]:
crm_network.reset()
op1 = crm_network.forward([1,2,3,4,5,6,7,8])
op1.shape

torch.Size([3, 1])

In [26]:
op2 = crm.forward([1,2,3,4,5,6,7,8])
op2.shape

TypeError: only integer tensors of a single element can be converted to an index

In [19]:
op = crm_network.forward_1([1,2,3,4,5,6,7,8])
op.shape

torch.Size([2, 1])

In [21]:
decoder.forward(op.reshape(1,-1))

In [9]:
crm_network.layerwise_neurons

{0: [0], 1: [1, 2], 2: [3, 4], 3: [5, 6, 7]}

In [22]:
crm_network.num_layers

4

In [15]:
[neuro for neuro in (crm_network.neurons) if neuro.layer == crm_network.num_layers-1]

 5: 0	 Grad: 0
             
 Predecessor: [3, 4]	Successor: [],
 6: 0	 Grad: 0
             
 Predecessor: [3, 4]	Successor: [],
 7: 0	 Grad: 0
             
 Predecessor: [3, 4]	Successor: []]

In [16]:
crm_network.topo_order

[0, 2, 1, 4, 3, 7, 6, 5]

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn

class rho_2_layer(nn.Module):
    num_layers = 0
    def __init__(self, layer_input_size, layer_output_size, model_input_size, neuron_mask_0, neuron_mask_prev):
        super(rho_2_layer, self).__init__()
        rho_2_layer.num_layers += 1
        self.layer_id = rho_2_layer.num_layers
        self.layer_0 = nn.Linear(model_input_size, layer_output_size, bias=False)
        self.layer_prev = nn.Linear(layer_input_size, layer_output_size, bias=False)
        self.activation = nn.ReLU()
        self.neuron_mask_0 = neuron_mask_0
        self.neuron_mask_prev = neuron_mask_prev
        self.bias = torch.nn.Parameter(torch.zeros(layer_output_size,))
                
    def forward(self, x, input_layer):
        self.layer_0.weight = torch.nn.Parameter(self.layer_0.weight * self.neuron_mask_0)
        self.layer_prev.weight = torch.nn.Parameter(self.layer_prev.weight * self.neuron_mask_prev)
        
        layer_out_0 = self.layer_0(input_layer) 
        layer_out_prev = self.layer_prev(x)
        activation_out = self.activation(layer_out_0 + layer_out_prev + self.bias)
        return activation_out
    
class rho_1_layer(nn.Module):
    num_layers = 0
    def __init__(self, layer_input_size, layer_output_size, neuron_mask_prev):
        super(rho_1_layer, self).__init__()
        rho_1_layer.num_layers += 1
        self.layer_id = rho_1_layer.num_layers
        self.layer_prev = nn.Linear(layer_input_size, layer_output_size, bias=False)
        self.activation = nn.ReLU()
        self.neuron_mask_prev = neuron_mask_prev
        self.bias = torch.nn.Parameter(torch.zeros(layer_output_size,))
                
    def forward(self, x):
        self.layer_prev.weight = torch.nn.Parameter(self.layer_prev.weight * self.neuron_mask_prev)
        
        layer_out_prev = self.layer_prev(x)
        activation_out = self.activation(layer_out_prev + self.bias)
        return activation_out

In [94]:
class CRM_model(nn.Module):
    def __init__(self, num_neurons, adj_list, d_rho1, d_rho2):
        super(CRM_model, self).__init__()
        self.num_neurons = num_neurons
        self.adj_list = adj_list
        self.adj_matrix = self._get_adjacency_matrix()
        self.layers_list = self._get_layers_list()
        #self.layers_list = layers_list
        self.d_rho1 = d_rho1
        self.d_rho2 = d_rho2
        self.input_size = len(self.layers_list[0])
        self.output_size = len(self.layers_list[-1])
        
        # print(self.adj_list, "\n")
        # print(self.adj_matrix, "\n")
        # print(self.layers_list, "\n")
        
        self.layers = self._setup_layers()
        
    def forward(self, inputs):
        input_layerwise = [x[..., neurons].detach().clone() for neurons in self.layers_list]
        input_layer = input_layerwise[0]
        x = inputs[..., self.layers_list[0]]
        for i, layer in enumerate(self.layers):
            if i < self.d_rho2:
                x = layer(x, input_layer) 
            else:
                x = layer(x)
                
    def _get_layers_list(self):
        layer_list = []
        prev_neurons = set()
        current_layer_neurons = []
        last_layer = []
        for i in self.adj_list:
            if self.adj_list[i] == []:
                last_layer.append(i)
            
            else:
                if i in prev_neurons:
                    prev_neurons = set()
                    layer_list.append(current_layer_neurons)
                    current_layer_neurons = []
                
                current_layer_neurons.append(i)
                prev_neurons = prev_neurons.union(set(self.adj_list[i]))
        layer_list.append(current_layer_neurons)
        layer_list.append(last_layer)

        return layer_list
        
    def _get_adjacency_matrix(self):
        # Create adjacency matrix using an adjacency list
        adj_matrix = torch.zeros(self.num_neurons, self.num_neurons)
        for i in range(self.num_neurons):
            if self.adj_list[i]:
                adj_matrix[i][self.adj_list[i]] = 1
        return adj_matrix
    
    def _get_submask(self, layer_in, layer_out):
        return self.adj_matrix[self.layers_list[layer_out]][:, self.layers_list[layer_in]]
    
        
    def _set_neuron_masks(self):
        # Create neuron masks for each layer based on the adjacency matrix
        neuron_masks = []
        for i in range(self.num_neurons):
            neuron_masks.append(self.adj_matrix[i])
        return neuron_masks
    
    def _setup_layers(self):
        # Creates a list of all layers
        layers = []
        for i in range(self.d_rho2 + self.d_rho1):
            if i < self.d_rho2:
                layers.append(rho_2_layer(len(self.layers_list[i]), 
                                          len(self.layers_list[i+1]), 
                                          self.input_size,
                                          self._get_submask(0, i+1),
                                          self._get_submask(i, i+1)
                                          ))
            
            else:
                layers.append(rho_1_layer(len(self.layers_list[i]),
                                          len(self.layers_list[i+1]), 
                                          self._get_submask(i, i+1)
                                          ))
        return layers

In [95]:
batch_size = 1
input_size = 16

num_neurons = 16
adj_list = {0: [3, 4, 9], 1: [4, 5], 2: [5, 3], 3: [6, 8, 9], 4: [6, 7], 5: [7, 8], 6: [10], 7: [11],
            8: [12], 9: [13], 10: [14], 11: [14], 12: [15], 13: [15], 14: [], 15: []}

# Create an instance of the CRM_model
crm_model = CRM_model(num_neurons, adj_list, d_rho1=2, d_rho2=2)

# Generate some input data
input_data = torch.randn(batch_size, input_size)

# Pass the input data through the model
output = crm_model(input_data)

# Print the output
print(output)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x16 and 3x3)

In [92]:
a = torch.rand( 8)
a

tensor([0.5470, 0.1542, 0.5988, 0.3392, 0.1366, 0.8948, 0.1371, 0.0651])

In [93]:
a[..., [1,2,3]]

tensor([0.1542, 0.5988, 0.3392])

In [33]:
adj_list = {0: [3, 4, 9], 1: [4, 5], 2: [5, 3], 3: [6, 8, 9], 4: [6, 7], 5: [7, 8], 6: [10], 7: [11],
            8: [12], 9: [13], 10: [14], 11: [14], 12: [15], 13: [15], 14: [], 15: []}
num_neurons = len(adj_list)

layer_list = []
prev_neurons = set()
current_layer_neurons = []
last_layer = []
for i in adj_list:
    if adj_list[i] == []:
        last_layer.append(i)
    
    else:
        if i in prev_neurons:
            prev_neurons = set()
            layer_list.append(current_layer_neurons)
            current_layer_neurons = []
        
        current_layer_neurons.append(i)
        prev_neurons = prev_neurons.union(set(adj_list[i]))
layer_list.append(current_layer_neurons)
layer_list.append(last_layer)

print(layer_list)

[[0, 1, 2], [3, 4, 5], [6, 7, 8, 9], [10, 11, 12, 13], [14, 15]]


In [54]:
mask_0 = torch.rand(4,3) > 0.9
mask_prev = torch.rand(4,3) > 0.9

In [55]:
mask_0, mask_prev

(tensor([[False, False, False],
         [False, False,  True],
         [False, False, False],
         [False, False, False]]),
 tensor([[False,  True, False],
         [False, False, False],
         [False, False, False],
         [False, False, False]]))

In [58]:
mask_prev[[1,2,3]][:,[2,1]]

tensor([[False, False],
        [False, False],
        [False, False]])

In [106]:
layer = rho_1_layer(3, 4, 3, mask_0, mask_prev)
layer(torch.tensor([1,2,3], dtype=torch.float32), torch.tensor([1,2,3], dtype=torch.float32))

tensor([0.0000, 0.0000, 0.0000, 0.1723], grad_fn=<ReluBackward0>)

In [90]:
layer.layer_0.weight.size()

torch.Size([4, 3])

In [4]:
import torch
torch.tensor([torch.tensor(i) for i in range(10)]).numpy()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)